In [ ]:
import os
import re
import io
import contextlib
import pandas as pd
from google import genai
from src.libs.lib import * 
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image

# 1. Credenciais e projeto
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ".formal-purpose-354320-af3d391f5234.json"
PROJECT_ID = "formal-purpose-354320"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# 2. Inicializa o client GenAI
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
MODEL_ID = "gemini-2.0-flash-001"

vertexai.init(project=PROJECT_ID, location=LOCATION)
# generative_multimodal_model = GenerativeModel("gemini-2.0-flash-thinking-exp-01-21")
generative_multimodal_model = GenerativeModel("gemini-2.0-flash-lite")



In [ ]:
prompt = f"""
Instruções: 
Você é um analista de dados e deve analisar os dados fornecidos de forma iterativa, 
passo a passo. Você analisa apenas numeros e textos. 
Não gera visualizações. 
Dese fazer pequenos passos de análise e não gerar respostas longas.
Deve enter o problema por partes e de forma ciclica.
Deve ter certeza que está seguindo o caminho correto antes de seguir para o próximo passo.
Não pular etapas mesmo que o processo fique longo.
"""
chat = generative_multimodal_model.start_chat()
response = chat.send_message(prompt)
show(response.text)

In [ ]:
caminho_dos_dados = "src/data/data.csv"
caminho_dos_objetivos = 'src/data/objetivos.txt'

In [ ]:
df = pd.read_csv(caminho_dos_dados, encoding='utf-8')
buf = io.StringIO()
df.info(buf=buf)
info_str = buf.getvalue()
describe_str = df.describe(include='all').to_string()
data = f"""
Aqui estão as características do meu dataset
Caminho do arquivo: {caminho_dos_dados}
INFO:
{info_str}
DESCRIBE:
{describe_str}
"""
show(data)

prompt = f"""
DADOS: {data}
Você deve entender os dados antes de começar.
"""
chat = generative_multimodal_model.start_chat()
response = chat.send_message(prompt)
show(response.text)

In [ ]:
with open(caminho_dos_objetivos, 'r', encoding='utf-8') as f:
    objetivo = f.read()
show(objetivo)
prompt = f"""
OBJETIVO: {objetivo}
Você deve entender o objetivo antes de começar.

"""
chat = generative_multimodal_model.start_chat()
response = chat.send_message(prompt)
show(response.text)

In [ ]:

def run_model(objective = None):
    history = ""
    # if objective is None:
    #     objective = "Continuar a partir do novo objetivo."

    prompt = f"""
    DADOS: {data}
    1) ENTENDER O OBJETIVO
    Interprete pŕoximo objetivo objetivo.
    """
    response = chat.send_message(prompt)
    show(response.text)


    prompt= f"""
        2) GERAR SCRIPT PYTHON
        URL do arquivo: "src/data/data.csv"
        Para atingir o objetivo a partir dos dados presentes no CSV:
        Gere um script simples que retorne um relatório em texto.
        Adicione todos os imports necessários.
        Não inclua visualizações.
        Não gerar imagens. 
        Gerar apenas scrips que retornem relatórios em texto que possam ser interpretados por IA posteriormente.
        Retornar apenas o código python
        Se for uma função, já chamar a função no script.
        Exemplo:
        def funcao(): #Definir a função
            return 0
        funcao() # Chamar a função sempre! 
        nstalar o pacote usando !pip install <pacote>
        """
    response = chat.send_message(prompt)    
    history += response.text
    show(response.text)

    import traceback
    import re
    raw_string = response.text
    match = re.search(r"```python\n(.*?)```", raw_string, re.DOTALL)
    code = match.group(1) if match else None
    # print(code)
    buffer = io.StringIO()
    try:
        with contextlib.redirect_stdout(buffer):
            exec(code, globals())
        code_output = buffer.getvalue()
    except Exception as e:
        # Captura erro e converte para string formatada
        code_output = f"⚠️ ERRO AO EXECUTAR O CÓDIGO GERADO:\n\n{traceback.format_exc()}"
    
    filename = f"src/data/output/script_{i}.py"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(code)
    history += response.text
    MAX_LEN = 1000
    if len(code_output) > MAX_LEN:
        code_output = code_output[:MAX_LEN] + (
            """\n\n<b style="color: red;">ATENÇÃO! A saída do código é muito grande,
            faça códigos que retornem saídas descritivas com no máximo 1000 caracteres. 
            Evite relatórios completos.</b>"""
        )
    show(code_output)


    prompt = f"""
    INTERPRETAR A SAIDA
    {code_output}
    Entender como a saída se relaciona com o objetivo de entrada e o que pode ser tirado disso 
    """

    response = chat.send_message(prompt)    
    history += response.text
    show(response.text)


    prompt = f"""
    DEFINIR O NOVO OBJETIVO 
    Definir o próximo objetivo simples que visa aprofundar a análise e se aproximar do objetivo primário. Este objetivo será usado como entrada no próximo ciclo. 
    Seja direto e sussinto.
    """
    response = chat.send_message(prompt)
    objective = response.text
    history += response.text
    show(response.text)

    filename = f"src/data/output/history_{i}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(history)

In [7]:
import time

for i in range(10):
    show(f"<h1>PASSO {i}</h1>")
    run_model()
    print(f"Aguardando 60 segundos antes do próximo passo ({i+1}/10)...")
    time.sleep(60)


Aguardando 60 segundos antes do próximo passo (2/10)...


<string>:119: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


Aguardando 60 segundos antes do próximo passo (3/10)...


Aguardando 60 segundos antes do próximo passo (4/10)...


<string>:84: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


...

Aguardando 60 segundos antes do próximo passo (6/10)...


<string>:99: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


Aguardando 60 segundos antes do próximo passo (7/10)...


<string>:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<string>:182: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.


ServiceUnavailable: 503 GOAWAY received; Error code: 0; Debug Text: session_timed_out